In [9]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import shap 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
import warnings

# I am adding thbis line to ignore all the futurewarnings that show up when i run the code.
# Which can clutter the final output
warnings.simplefilter(action='ignore', category=FutureWarning)


def display(df):
    print(df.to_string())


#Let's Load the Data
try:
    fileName = "Pred_Ast_Diam_2.csv"
    df = pd.read_csv(f'../data/{fileName}')
except FileNotFoundError:
    print(f"Could not find the {fileName}")
    df = pd.DataFrame()
if not df.empty:
    print("Dataset Loaded Successfully")
    print(df.head)
    print(df.shape)


Dataset Loaded Successfully
<bound method NDFrame.head of        orbit_id         e         a          i          om           w  \
0        JPL 35  0.242027  2.201791   2.536221  313.311389   18.989048   
1        JPL 25  0.256856  2.338209  22.326589   10.489602  105.115594   
2        JPL 28  0.160543  2.228812   1.747387  121.579382  252.465454   
3        JPL 35  0.167945  2.241299   2.428619  161.636895  172.846491   
4        JPL 34  0.253295  2.467536   6.757106  137.130656  259.158793   
...         ...       ...       ...        ...         ...         ...   
126492    JPL 4  0.430349  3.155975  28.718353  115.532995  136.849398   
126493    JPL 8  0.159119  3.171225  27.098625  309.036573   19.746812   
126494    JPL 7  0.076071  2.548410  11.593237  246.298656  170.090810   
126495    JPL 8  0.287449  3.051336  14.456779  343.917822  342.614839   
126496    JPL 5  0.109001  2.417477   4.525668  148.244819   31.949854   

                ma         n            tp      moid 